In [31]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import requests
from datetime import datetime

# Ticket Melon

In [43]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac-119/chromedriver'
chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)

driver = webdriver.Chrome(service=chrome_driver_service)
start_time = time.time()
driver.get("https://www.ticketmelon.com")

wait = WebDriverWait(driver, 10) #timeout=20 sec

# start_time = time.time()
# elementwait = wait.until(EC.presence_of_element_located((By.ID, "__next")))
# end_time = time.time()
# wait_time = end_time - start_time 
# print("waited for \"__next\" ", wait_time, "seconds")

scroll_distance = 1200
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

end_time = time.time()
wait_time = end_time - start_time 
print("waited", wait_time, "sec for first scroll")
print("scroll to entertainment :" , scroll_distance, "px")

entertainment_button = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.slick-slide.slick-active.slick-current')))
entertainment_button.click()

scroll_distance = 1500
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to see more :" , scroll_distance, "px")

see_more_button = driver.find_element(
    By.CSS_SELECTOR, '.HomeStyled__EventSeeMore-sc-125xn4a-2.eMhTTB')
driver.execute_script("arguments[0].click();", see_more_button)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

waited 32.83982706069946 sec for first scroll
scroll to entertainment : 1200 px
scroll to see more : 1500 px


In [44]:
elements = soup.find_all('div',
                         class_='MelonEventCardStyled__EventDetail-sc-8kc3ju-2'
                         )  #event-date, event-name, event-location
event_data = []
for element in elements:
    event_info = {
        "title": element.find('p', class_='event-name').text,
        "date": element.find('p', class_='event-date').text,
        "location": element.find('p', class_='event-location').text
    }
    event_data.append(event_info)
melon_concert = pd.DataFrame(event_data)


## modify date

In [45]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip() 
melon_concert[['startdate', 'enddate']] = melon_concert['date'].apply(extract_dates).apply(pd.Series)

display(melon_concert)

,title,date,location,startdate,enddate
0,808 Festival 2023 - GA AND VIP STANDING TICKETS,8 Dec - 10 Dec,BITEC Bangna,8 Dec,10 Dec
1,Sunrise Brew Yoga on Cruise,2 Dec,River city bangkok,2 Dec,2 Dec
2,PELUPO 2024,1 Mar - 2 Mar,"The Fields, Siam Country Club, Pattaya",1 Mar,2 Mar
3,UnKonscious Festival 2024,30 Jan - 4 Feb,Alexa Beach club,30 Jan,4 Feb
4,Monet and Friends Alive Bangkok,22 Sep - 7 Jan,"Attraction Hall, 6th Floor, ICONSIAM",22 Sep,7 Jan
...,...,...,...,...,...
146,Subculture Thailand 2024 - Weekender,1 Mar - 3 Mar,Bangkok,1 Mar,3 Mar
147,Citizen Live In Bangkok 2024,10 Mar,Mr.fox,10 Mar,10 Mar
148,SLASH featuring Myles Kennedy Live in Bangkok,15 Mar,CENTERPOINT STUDIO,15 Mar,15 Mar
149,Southside Festival,30 Mar,Andamanda Phuket,30 Mar,30 Mar


In [46]:
melon_concert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      151 non-null    object
 1   date       151 non-null    object
 2   location   151 non-null    object
 3   startdate  151 non-null    object
 4   enddate    151 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [47]:
def calculate_year(date_str):
    # Convert month abbreviations to numeric values
    month_values = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    # Extract the month from the date
    month = date_str.split()[1]
    
    if month_values.get(month):
        if month_values[month] >= 6:
            return 2023
        else:
            return 2024
    else:
        return None  # Handle invalid month

melon_concert['year'] = melon_concert['startdate'].apply(calculate_year)

melon_concert.head(3)

,title,date,location,startdate,enddate,year
0,808 Festival 2023 - GA AND VIP STANDING TICKETS,8 Dec - 10 Dec,BITEC Bangna,8 Dec,10 Dec,2023
1,Sunrise Brew Yoga on Cruise,2 Dec,River city bangkok,2 Dec,2 Dec,2023
2,PELUPO 2024,1 Mar - 2 Mar,"The Fields, Siam Country Club, Pattaya",1 Mar,2 Mar,2024


In [48]:
melon_concert['year'] = melon_concert['year'].astype(str)
melon_concert['startdate'] = melon_concert['startdate'] + ' ' + melon_concert['year']
melon_concert['startdate'] = pd.to_datetime(melon_concert['startdate'], format='%d %b %Y', errors='coerce')

melon_concert['enddate'] = melon_concert['enddate'] + ' ' + melon_concert['year']
melon_concert['enddate'] = pd.to_datetime(melon_concert['enddate'], format='%d %b %Y', errors='coerce')

In [49]:
melon_concert.head(3)

,title,date,location,startdate,enddate,year
0,808 Festival 2023 - GA AND VIP STANDING TICKETS,8 Dec - 10 Dec,BITEC Bangna,2023-12-08,2023-12-10,2023
1,Sunrise Brew Yoga on Cruise,2 Dec,River city bangkok,2023-12-02,2023-12-02,2023
2,PELUPO 2024,1 Mar - 2 Mar,"The Fields, Siam Country Club, Pattaya",2024-03-01,2024-03-02,2024


## liite touch.

In [50]:
melon_concert.drop(columns=['year'], inplace=True)
melon_concert = melon_concert.sort_values(by=['startdate', 'enddate'])

In [51]:
melon_concert.drop(columns=['date'], inplace=True)

In [52]:
display(melon_concert)

,title,location,startdate,enddate
4,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",2023-09-22,2023-01-07
5,ZEENUNEW 1st CONCERT ANOTHER LIFE | DVD BOXSET,Photobook Delivery,2023-10-31,2023-12-01
6,Enchanted Flora Park,Flora Park,2023-11-01,NaT
7,BEACH CLUB CULTURE November Weekends at Alexa ...,Alexa Beach Club Pattaya,2023-11-04,2023-11-26
10,MOB PARTY 23,UNDERGROUND CHIANGMAI NIGHT BAZAAR,2023-11-25,2023-11-25
...,...,...,...,...
146,Subculture Thailand 2024 - Weekender,Bangkok,2024-03-01,2024-03-03
147,Citizen Live In Bangkok 2024,Mr.fox,2024-03-10,2024-03-10
148,SLASH featuring Myles Kennedy Live in Bangkok,CENTERPOINT STUDIO,2024-03-15,2024-03-15
149,Southside Festival,Andamanda Phuket,2024-03-30,2024-03-30


In [53]:
melon_concert['vendor'] = 'Ticketmelon'

In [54]:
nan_rows = melon_concert.loc[melon_concert.isna().any(axis=1)]
print(nan_rows)

                  title    location  startdate enddate       vendor
6  Enchanted Flora Park  Flora Park 2023-11-01     NaT  Ticketmelon


In [55]:
# oct_con = melon_concert[melon_concert['startdate'].dt.month == 10]
# display(oct_con)

# TTM

## make df

In [56]:
TTMbigdata = requests.get("https://www.thaiticketmajor.com/concert/")

In [57]:
TTMsoup = BeautifulSoup(TTMbigdata.text, "html.parser")
TTMsoup.find_all("div", class_="box-txt")

div_elements = TTMsoup.find_all("div", class_="box-txt")
event_data = []

for div in div_elements:
    event_info = {
        "title": div.find("a", class_="title").text if div.find("a", class_="title") else "",
        "date": div.find("span", class_="datetime").text if div.find("span", class_="datetime") else "",
        "location": div.find("a", class_="venue").text if div.find("a", class_="venue") else ""
    }
    event_data.append(event_info)


ttm_concert = pd.DataFrame(event_data)
ttm_concert = ttm_concert[~ttm_concert['date'].str.contains('Public Sale')]
ttm_concert.head(5)

,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 ม.ค. 2567 - 21 ม.ค. 2567,อิมแพ็ค อารีน่า เมืองทองธานี
1,HIKING BECKY DAY FAN CAMPING,24 ธ.ค. 2566,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 ม.ค. 2567,อิมแพ็ค อารีน่า เมืองทองธานี
3,กาลครั้ง 5 TATTOO COLOUR FEST,3 ก.พ. 2567,อิมแพ็ค อารีน่า เมืองทองธานี
4,Tom Jones : Ages & Stage Tour - Bangkok,14 มี.ค. 2567,ยูโอบี ไลฟ์


## modify date

In [58]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

# Apply the replacement function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(replace_thai_month_with_english)
ttm_concert.head(1)

,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2567 - 21 Jan 2567,อิมแพ็ค อารีน่า เมืองทองธานี


In [59]:
ttm_concert.reset_index(drop=True, inplace=True)
display(ttm_concert)

,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2567 - 21 Jan 2567,อิมแพ็ค อารีน่า เมืองทองธานี
1,HIKING BECKY DAY FAN CAMPING,24 Dec 2566,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 Jan 2567,อิมแพ็ค อารีน่า เมืองทองธานี
3,กาลครั้ง 5 TATTOO COLOUR FEST,3 Feb 2567,อิมแพ็ค อารีน่า เมืองทองธานี
4,Tom Jones : Ages & Stage Tour - Bangkok,14 Mar 2567,ยูโอบี ไลฟ์
5,ม่วนจอย ลอยบน Cloud BE ON CLOUD NEW YEAR COUNT...,31 Dec 2566,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
6,"Pictured Resort 1st Live in Bangkok "" Away To...",26 Nov 2566,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน
7,Music @ Mahidol,29 Nov 2566,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา)
8,พรหมลิขิต Fan Meeting จะชาตินี้ชาติไหนหัวใจก็ค...,30 Nov 2566,"TRUE ICON HALL, 7th FLOOR, ICONSIAM"
9,DISNEY PRINCESS - THE CONCERT IN BANGKOK,2 Dec 2566 - 3 Dec 2566,เมืองไทยรัชดาลัย เธียเตอร์


In [60]:
# ttm_concert.loc[1,['date']] = '23 Dec 2566'
# rows = [7, 35, 39, 40]
# ttm_concert.drop(rows, inplace=True)

In [61]:
# Function to subtract 543 from the year part of a date string
def subtract_543_from_year(date_str):
    # Split the date string by spaces
    parts = date_str.split()
    
    # Check if the date string has the format "DD MMM YYYY - DD MMM YYYY"
    if len(parts) == 7:
        # Extract the year parts
        year_parts = parts[2], parts[-1]
        
        # Subtract 543 from each year part
        updated_year_parts = [str(int(year) - 543) for year in year_parts]
        
        # Replace the year parts in the date string
        parts[2], parts[-1] = updated_year_parts
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str
    else:
        # Extract the year part
        year_part = parts[-1]
        
        # Subtract 543 from the year part
        updated_year_part = str(int(year_part) - 543)
        
        # Replace the year part in the date string
        parts[-1] = updated_year_part
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str

# Apply the subtraction function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(subtract_543_from_year)

ttm_concert.head(3)


,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2024 - 21 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
1,HIKING BECKY DAY FAN CAMPING,24 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี


In [69]:
display(ttm_concert)

,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2024 - 21 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
1,HIKING BECKY DAY FAN CAMPING,24 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
3,กาลครั้ง 5 TATTOO COLOUR FEST,3 Feb 2024,อิมแพ็ค อารีน่า เมืองทองธานี
4,Tom Jones : Ages & Stage Tour - Bangkok,14 Mar 2024,ยูโอบี ไลฟ์
5,ม่วนจอย ลอยบน Cloud BE ON CLOUD NEW YEAR COUNT...,31 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
6,"Pictured Resort 1st Live in Bangkok "" Away To...",26 Nov 2023,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน
7,Music @ Mahidol,29 Nov 2023,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา)
8,พรหมลิขิต Fan Meeting จะชาตินี้ชาติไหนหัวใจก็ค...,30 Nov 2023,"TRUE ICON HALL, 7th FLOOR, ICONSIAM"
9,DISNEY PRINCESS - THE CONCERT IN BANGKOK,2 Dec 2023 - 3 Dec 2023,เมืองไทยรัชดาลัย เธียเตอร์


### clean

In [70]:
ttm_concert = ttm_concert.drop(28)

In [71]:
display(ttm_concert)

,title,date,location
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2024 - 21 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
1,HIKING BECKY DAY FAN CAMPING,24 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
3,กาลครั้ง 5 TATTOO COLOUR FEST,3 Feb 2024,อิมแพ็ค อารีน่า เมืองทองธานี
4,Tom Jones : Ages & Stage Tour - Bangkok,14 Mar 2024,ยูโอบี ไลฟ์
5,ม่วนจอย ลอยบน Cloud BE ON CLOUD NEW YEAR COUNT...,31 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์"
6,"Pictured Resort 1st Live in Bangkok "" Away To...",26 Nov 2023,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน
7,Music @ Mahidol,29 Nov 2023,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา)
8,พรหมลิขิต Fan Meeting จะชาตินี้ชาติไหนหัวใจก็ค...,30 Nov 2023,"TRUE ICON HALL, 7th FLOOR, ICONSIAM"
9,DISNEY PRINCESS - THE CONCERT IN BANGKOK,2 Dec 2023 - 3 Dec 2023,เมืองไทยรัชดาลัย เธียเตอร์


In [72]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3}\s*\d{4})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip()

ttm_concert[['startdate', 'enddate']] =ttm_concert['date'].apply(extract_dates).apply(pd.Series)
ttm_concert.head(3)

,title,date,location,startdate,enddate
0,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2024 - 21 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี,20 Jan 2024,21 Jan 2024
1,HIKING BECKY DAY FAN CAMPING,24 Dec 2023,"ยูเนี่ยน ฮอลล์, ศูนย์การค้ายูเนี่ยน มอลล์",24 Dec 2023,24 Dec 2023
2,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,27 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี,27 Jan 2024,27 Jan 2024


In [ ]:
def convert_date_format(date_str):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_str, '%d %b %Y')
    
    # Format the datetime object as 'YYYY-MM-DD'
    formatted_date = date_obj.strftime('%Y-%m-%d')
    
    return formatted_date

ttm_concert['startdate'] = ttm_concert['startdate'].apply(convert_date_format)
ttm_concert['enddate'] = ttm_concert['enddate'].apply(convert_date_format)

ttm_concert.head()

,title,date,location,startdate,enddate
0,Ed Sheeran ‘+ - = ÷ x' Mathematics Tour Bangko...,10 Feb 2024,สนามราชมังคลากีฬาสถาน,2024-02-10,2024-02-10
1,GMMTV STARLYMPIC 2023,23 Dec 2023,อิมแพ็ค อารีน่า เมืองทองธานี,2023-12-23,2023-12-23
2,เฉลียง Rare Item RESTAGE คอนเสิร์ต,9 Dec 2023 - 10 Dec 2023,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-09,2023-12-10
3,"Daniel Caesar Presents ""Superpowers World Tour...",9 Dec 2023,"อิมแพ็ค เอ็กซิบิชั่น ฮอลล์ 5, เมืองทองธานี",2023-12-09,2023-12-09
4,RS MUSIC ร่วมกับ อำพลฟูดส์ Present CONCERT SHO...,16 Dec 2023,อิมแพ็ค อารีน่า,2023-12-16,2023-12-16


In [ ]:
# ttm_concert.astype(da)

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 5 to 152
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      153 non-null    object        
 1   location   153 non-null    object        
 2   startdate  153 non-null    datetime64[ns]
 3   enddate    153 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 6.0+ KB


In [73]:
ttm_concert['startdate'] = pd.to_datetime(ttm_concert['startdate'])
ttm_concert['enddate'] = pd.to_datetime(ttm_concert['enddate'])
ttm_concert = ttm_concert.sort_values(['startdate', 'enddate'])
ttm_concert = ttm_concert.drop(columns=['date'], axis=1)
display(ttm_concert)

,title,location,startdate,enddate
6,"Pictured Resort 1st Live in Bangkok "" Away To...",โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน,2023-11-26,2023-11-26
7,Music @ Mahidol,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา),2023-11-29,2023-11-29
8,พรหมลิขิต Fan Meeting จะชาตินี้ชาติไหนหัวใจก็ค...,"TRUE ICON HALL, 7th FLOOR, ICONSIAM",2023-11-30,2023-11-30
11,SoundAbout Mountain Fest,Midwinter เขาใหญ่,2023-12-02,2023-12-02
12,Young K Solo Concert < Letters with notes > in...,"แจ้งวัฒนะ ฮอลล์, เซ็นทรัลแจ้งวัฒนะ ชั้น 5.5",2023-12-02,2023-12-02
13,STAR FOR YOU MEETING CONCERT,"Ballroom Hall 1-4, ศูนย์ประชุมแห่งชาติสิริกิติ์",2023-12-02,2023-12-02
9,DISNEY PRINCESS - THE CONCERT IN BANGKOK,เมืองไทยรัชดาลัย เธียเตอร์,2023-12-02,2023-12-03
10,Thailand Phil 2023/24 Season,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07
14,"Yama 1st ASIA TOUR ""thanks for waiting""",The Portal Ballroom (Muang Thong Thani),2023-12-03,2023-12-03
15,Jay Chou Carnival World Tour 2023 ' Bangkok,สนามราชมังคลากีฬาสถาน,2023-12-08,2023-12-09


In [74]:
ttm_concert['vendor'] = 'TTM'

# The Concert

In [78]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac-119/chromedriver'
chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_driver_service)
driver.get("https://www.theconcert.com/concert")

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

last_height = 0
print(last_height)
time.sleep(5)
while(True) : 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if new_height == last_height :
        break 
    else :
        last_height = new_height

wait = WebDriverWait(driver, 10)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

0
1291
2337
4788
4788


In [79]:
elements = soup.find_all('div',
                         class_='recom-detail-box'
                         )  #conert-recom-title, concert-recom-date, concert-recom-date, concert-recom-local
event_data = []
for element in elements:
    event_info = {
        "title": element.find('div', class_='concert-recom-title').text,
        "date": element.find('div', class_='concert-recom-date').text,
        "location": element.find('div', class_='concert-recom-local').text
    }
    event_data.append(event_info)

the_concert = pd.DataFrame(event_data)
the_concert.head(3)

,title,date,location
0,People Fest '23 Countdown Concert Zeal x Klear,22 ต.ค. - 31 ธ.ค. 23,PEOPLE FEST
1,E~San 11th Music Festival Thailand 2023,24 - 26 พ.ย. 23,หาดสวรรค์ เขื่อนอุบลรัตน์ จ.ขอนแก่น
2,COUCH LOCK THAILAND 2023,25 - 26 พ.ย. 23,ESC Park Rangsit


## mo date

In [80]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

the_concert['date'] = the_concert['date'].apply(replace_thai_month_with_english)
display(the_concert)


,title,date,location
0,People Fest '23 Countdown Concert Zeal x Klear,22 Oct - 31 Dec 23,PEOPLE FEST
1,E~San 11th Music Festival Thailand 2023,24 - 26 Nov 23,หาดสวรรค์ เขื่อนอุบลรัตน์ จ.ขอนแก่น
2,COUCH LOCK THAILAND 2023,25 - 26 Nov 23,ESC Park Rangsit
3,est Cola Presents Monster Music Festival 2023,25 - 26 Nov 23,สนามกีฬาแห่งชาติและสนามเทพหัสดิน
4,[LIVE STREAMING] ANKNOWNA The Unexpected Expe...,26 Nov 23,MCC HALL THE MALL LIFESTORE NGAMWONGWAN
5,ANKNOWNA The Unexpected Experiment ( DAY 2),26 Nov 23,MCC HALL THE MALL LIFESTORE NGAMWONGWAN
6,Longlay Beach Life Festival #4,2 - 3 Dec 23,Diamond Beach (Longlay Beach)
7,Maho Rasop 2023,2 - 3 Dec 23,ESC PARK BANGKOK
8,รถตู้รับส่งงาน Maho Rasop 2023,2 - 3 Dec 23,ESC PARK BANGKOK
9,Journey Fest | เจอกันนี่ เฟส เชียงใหม่,2 - 3 Dec 23,Journey Fest At The Consul's Garden


In [81]:
def change_year(date_str):
    # Use regular expression to find and replace the year
    updated_date_str = re.sub(r'\b23$', '2023', date_str)
    updated_date_str = re.sub(r'\b24$', '2024', updated_date_str)
    return updated_date_str

the_concert['date'] = the_concert['date'].apply(change_year)
the_concert.head(1)

,title,date,location
0,People Fest '23 Countdown Concert Zeal x Klear,22 Oct - 31 Dec 2023,PEOPLE FEST


In [82]:
def split_date(date_str):
    parts = date_str.split('-')
    start_date = parts[0].strip()
    
    if len(parts) > 1:
        end_date = parts[1].strip()
        end_day, end_month, year = end_date.split()
        try:
            start_day, start_month = start_date.split()
            print(start_day, start_month)

        except ValueError:
            start_day = start_date
            start_month = end_month  # or any other suitable default value

        start_date = f"{start_day} {start_month} {year}"
    else:
        end_date = start_date
        
    return start_date, end_date
# Apply the function to the 'date' column
the_concert[['startdate', 'enddate']] = the_concert['date'].apply(split_date).apply(pd.Series)
the_concert.head(3)

22 Oct


,title,date,location,startdate,enddate
0,People Fest '23 Countdown Concert Zeal x Klear,22 Oct - 31 Dec 2023,PEOPLE FEST,22 Oct 2023,31 Dec 2023
1,E~San 11th Music Festival Thailand 2023,24 - 26 Nov 2023,หาดสวรรค์ เขื่อนอุบลรัตน์ จ.ขอนแก่น,24 Nov 2023,26 Nov 2023
2,COUCH LOCK THAILAND 2023,25 - 26 Nov 2023,ESC Park Rangsit,25 Nov 2023,26 Nov 2023


## little touch

In [83]:
the_concert = the_concert.drop(columns=['date'], axis=1)
the_concert['startdate'] = pd.to_datetime(the_concert['startdate'], format='%d %b %Y', errors='coerce')
the_concert['enddate'] = pd.to_datetime(the_concert['enddate'], format='%d %b %Y', errors='coerce')

the_concert = the_concert.sort_values(['startdate', 'enddate'])
the_concert.reset_index(drop=True, inplace=True)
display(the_concert)

,title,location,startdate,enddate
0,People Fest '23 Countdown Concert Zeal x Klear,PEOPLE FEST,2023-10-22,2023-12-31
1,E~San 11th Music Festival Thailand 2023,หาดสวรรค์ เขื่อนอุบลรัตน์ จ.ขอนแก่น,2023-11-24,2023-11-26
2,COUCH LOCK THAILAND 2023,ESC Park Rangsit,2023-11-25,2023-11-26
3,est Cola Presents Monster Music Festival 2023,สนามกีฬาแห่งชาติและสนามเทพหัสดิน,2023-11-25,2023-11-26
4,[LIVE STREAMING] ANKNOWNA The Unexpected Expe...,MCC HALL THE MALL LIFESTORE NGAMWONGWAN,2023-11-26,2023-11-26
5,ANKNOWNA The Unexpected Experiment ( DAY 2),MCC HALL THE MALL LIFESTORE NGAMWONGWAN,2023-11-26,2023-11-26
6,The Yers live at Come Again Cafe & Restaurant,Come Again Cafe & Restaurant,2023-11-29,2023-11-29
7,ZEROTOZOLAR 1st FANMEETING,Search Studio,2023-12-02,2023-12-02
8,รำวงเฟสติวัล,ลานพาร์คอป พาร์ค,2023-12-02,2023-12-02
9,GOLF-MIKE : BOUNCE TO THE FUTURE CONCERT,IMPACT ARENA,2023-12-02,2023-12-02


In [84]:
the_concert['vendor'] = 'The Concert'

In [85]:
nan_rows = the_concert.loc[the_concert.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [title, location, startdate, enddate, vendor]
Index: []


# Combined 

In [86]:
df = pd.concat([melon_concert, ttm_concert, the_concert])
df = df.sort_values(['startdate', 'enddate'])
df.reset_index(drop=True, inplace=True)
display(df)

,title,location,startdate,enddate,vendor
0,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",2023-09-22,2023-01-07,Ticketmelon
1,People Fest '23 Countdown Concert Zeal x Klear,PEOPLE FEST,2023-10-22,2023-12-31,The Concert
2,ZEENUNEW 1st CONCERT ANOTHER LIFE | DVD BOXSET,Photobook Delivery,2023-10-31,2023-12-01,Ticketmelon
3,Enchanted Flora Park,Flora Park,2023-11-01,NaT,Ticketmelon
4,BEACH CLUB CULTURE November Weekends at Alexa ...,Alexa Beach Club Pattaya,2023-11-04,2023-11-26,Ticketmelon
...,...,...,...,...,...
233,Tom Jones : Ages & Stage Tour - Bangkok,ยูโอบี ไลฟ์,2024-03-14,2024-03-14,TTM
234,Tom Jones : Ages & Stage Tour - Bangkok,ยูโอบี ไลฟ์,2024-03-14,2024-03-14,TTM
235,SLASH featuring Myles Kennedy Live in Bangkok,CENTERPOINT STUDIO,2024-03-15,2024-03-15,Ticketmelon
236,Southside Festival,Andamanda Phuket,2024-03-30,2024-03-30,Ticketmelon


In [87]:
path = '/Users/got/Documents/retune/concert_data.csv'
df.to_csv(path, index=False)